In [1]:
"""Created on Monday Jan 6 11:03:00 
2019 Group 2
@authors: Sibel Yuksel, Marianne Manson (v1)"""

'Created on Monday Jan 6 11:03:00 \n2019 Group 2\n@authors: Sibel Yuksel'

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import datetime
import json

In [12]:
df_scrap = pd.DataFrame(columns=['art_content','art_content_html','art_published_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','src_img','art_auth','art_tag']  )
df_scrap

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag


In [23]:
def scraping_lemonde(url : 'str'):
    """Documentation
    function which from a url creates a BeautifulSoup object, then extract different informations about the article and the 
    source. Then researches informations about the article and the website. It finally returns all this data as a dict
  
    Parameters:
        url(str): The url that we will scrap 
        
    Out:
        new_row: it contains some propreties of the article and the sources 

    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    
    src_type = 'xpath_source'
    src_url = 'https://www.lemonde.fr/'
    src_name = "Le Monde"
    
    # find the article URL (art_url) 
    if html_soup.find("meta", {"property": "og:url"}) is not None:
        art_url = html_soup.find("meta", {"property": "og:url"})['content']
    else:
        art_url = url
        
    # find the article Title (art_title)  
    art_title = html_soup.title.get_text().replace('\xa0','')
    
    
    # find the article Author (art_auth)  
    if html_soup.find("meta",{"property":"og:article:author"}) is not None:
        art_auth = html_soup.find("meta",{"property":"og:article:author"})['content']
        if art_auth==[]: art_auth = 'no-data' 
    else:
        art_auth = 'no_data'
        
        
    # find the date of publication of the article (art_published_datetime) (format: datetime)  
    if html_soup.find("meta", {"property":"og:article:published_time"}) is not None:
        art_published = html_soup.find("meta", {"property":"og:article:published_time"})['content'][:10]      # take the date and remove the hour
        art_published_datetime = datetime.datetime.strptime(art_published,'%Y-%m-%d').date()  # put at the format datetime      
    # if there is no date, we replace None with the date of today
    else:
        art_published_datetime = datetime.datetime.today().date()     # 

        
    #src_img  
    if html_soup.find("figure", {"class":"article__media"}) is not None:
        art_img = html_soup.find("figure", {"class":"article__media"}).find('img')['src']
    else:
        art_img = 'no_data'
        
        
    #art_tag  
    #art_tag = json.loads(html_soup.find('script', type = 'application/ld+json')['@type'])
    art_tag = "no_data"

        
    #art_content_html  and  art_content
    try:
        art_content_html_corps = html_soup.find("article", {"class": "article__content old__article-content-single"})
        if html_soup.find("p", {"class": "article__desc"}) is not None:
            art_content_html_intro = html_soup.find("p", {"class": "article__desc"}) #prend une sous balise en trop 
            art_content_html = [art_content_html_intro,art_content_html_corps]  
            art_content = (art_content_html_intro.get_text() + art_content_html_corps.get_text()).replace('\xa0','')
        else:
            art_content_html = art_content_html_corps
            art_content = art_content_html_corps.get_text().replace('\xa0','')
            # REPLACE NE MARCHE PAS
    except:
        #problems with https://www.lemonde.fr/transition-ecologique/article/2020/08/03/les-villes-et-leurs-jumeaux-numeriques_6048030_179.html
        art_content_html = np.nan
        art_content = np.nan

    #art_lang   
    art_lang = 'fr'
    
    #return art_content
    new_row = {'art_content': art_content ,
               'art_content_html': art_content_html ,
               'art_published_datetime': art_published_datetime ,
               'art_lang': art_lang , 
               'art_title' : art_title , 
               'art_url' : art_url ,
               'src_name' : src_name ,
               'src_type' : src_type ,
               'src_url' : src_url ,
               'src_img' : art_img ,
               'art_auth': art_auth ,
               'art_tag': art_tag }
    return new_row

In [25]:
#url1 = 'https://www.lemonde.fr/transition-ecologique/article/2020/08/03/les-villes-et-leurs-jumeaux-numeriques_6048030_179.html'
#url1 doesn't work
url2 = "https://www.lemonde.fr/pixels/article/2020/09/16/donnees-de-sante-nouveau-recours-contre-le-health-data-hub-devant-le-conseil-d-etat_6052464_4408996.html"
url3 = 'https://www.lemonde.fr/pixels/article/2020/03/20/contre-la-pandemie-de-nombreux-pays-misent-sur-la-surveillance-permise-par-le-big-data_6033851_4408996.html'
url4 = 'https://www.lemonde.fr/vie-quotidienne/article/2015/10/20/bruit-victoire-de-riverains-contre-un-data-center_6004076_5057666.html'

new_row = scraping_lemonde(url2)
df_test = df_scrap.append(new_row, ignore_index=True)
df_test

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
0,"Le Health Data Hub, lancé en décembre dern...","[[ Le Health Data Hub, lancé en décembre de...",2020-09-16,fr,Données de santé: nouveau recours contre le He...,https://www.lemonde.fr/pixels/article/2020/09/...,Le Monde,xpath_source,https://www.lemonde.fr/,no_data,no_data,no_data
